# Architectures

> Scripts to get architectures for the models

In [12]:
#| default_exp architectures

In [13]:
#| export
#| hide

import torch
import torch.nn as nn
from abc import ABC, abstractmethod
from torch import Tensor
from pytorch_lightning.callbacks import Callback
import matplotlib.pyplot as plt

## Sampling

In [14]:
#| export
class Sampling(nn.Module):
    def forward(self, z_mean: Tensor, z_log_var: Tensor) -> Tensor:
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std

## Callback

In [15]:
#| export
class VAELossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.train_total_losses = []
        self.train_recon_losses = []
        self.train_kl_losses = []
        self.val_total_losses = []
        self.val_recon_losses = []
        self.val_kl_losses = []

    def on_train_epoch_end(self, trainer, pl_module):
        train_total_loss = trainer.callback_metrics.get('train_total_loss')
        train_recon_loss = trainer.callback_metrics.get('train_reconstruction_loss')
        train_kl_loss = trainer.callback_metrics.get('train_kl_loss')

        if train_total_loss is not None:
            self.train_total_losses.append(train_total_loss.item())
        if train_recon_loss is not None:
            self.train_recon_losses.append(train_recon_loss.item())
        if train_kl_loss is not None:
            self.train_kl_losses.append(train_kl_loss.item())

    def on_validation_epoch_end(self, trainer, pl_module):
        val_total_loss = trainer.callback_metrics.get('val_total_loss')
        val_recon_loss = trainer.callback_metrics.get('val_reconstruction_loss')
        val_kl_loss = trainer.callback_metrics.get('val_kl_loss')

        if val_total_loss is not None:
            self.val_total_losses.append(val_total_loss.item())
        if val_recon_loss is not None:
            self.val_recon_losses.append(val_recon_loss.item())
        if val_kl_loss is not None:
            self.val_kl_losses.append(val_kl_loss.item())

    def plot_total_losses(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_total_losses, label='Training Total Loss')
        plt.plot(self.val_total_losses, label='Validation Total Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Total Loss')
        plt.title('Total Training and Validation Losses')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_all_losses(self):
        plt.figure(figsize=(15, 10))
        
        # Plot total losses
        plt.subplot(3, 1, 1)
        plt.plot(self.train_total_losses, label='Training Total Loss')
        plt.plot(self.val_total_losses, label='Validation Total Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Total Loss')
        plt.title('Total Training and Validation Losses')
        
        # Plot reconstruction losses
        plt.subplot(3, 1, 2)
        plt.plot(self.train_recon_losses, label='Training Reconstruction Loss')
        plt.plot(self.val_recon_losses, label='Validation Reconstruction Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Reconstruction Loss')
        
        # Plot KL divergence losses
        plt.subplot(3, 1, 3)
        plt.plot(self.train_kl_losses, label='Training KL Divergence Loss')
        plt.plot(self.val_kl_losses, label='Validation KL Divergence Loss')
        
        for i in range(1, 4):
            plt.subplot(3, 1, i)
            plt.legend()
            plt.grid(True)

        plt.tight_layout()
        plt.show()

## Conv5 Architecture

### Kernel Sizes: 5, 7 9, 13

In [16]:
#| export
class Conv5Encoder(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Flatten()
        )
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)
        )

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

In [17]:
#| export
class Conv5Decoder(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * self.seq_len),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, self.seq_len))
        )

        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding=6),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding=3),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, z: Tensor) -> Tensor:
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

In [18]:
#| export
def get_conv5_vae_components(seq_len, feat_dim, latent_dim, dropout_rate=0.2):
    """
    Returns an instance of Conv5Encoder and Conv5Decoder based on the given parameters.
    
    Args:
        seq_len (int): Length of input sequence.
        feat_dim (int): Dimensionality of input features.
        latent_dim (int): Dimensionality of the latent space.
        dropout_rate (float): Dropout rate to use in the model.
    
    Returns:
        encoder (Conv5Encoder): The encoder part of the VAE.
        decoder (Conv5Decoder): The decoder part of the VAE.
    """
    encoder = Conv5Encoder(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5Decoder(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

### Legit Tsgm

In [19]:
#| export
class Conv5EncoderLegitTsgm(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=10, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=4, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Flatten()
        )
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)  # Output z_mean and z_log_var
        )

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

In [20]:
#| export
class Conv5DecoderLegitTsgm(nn.Module):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate

        # Dense layers to upscale the latent dimensions
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * seq_len),  # Adjust output size based on sequence length
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, seq_len))
        )

        # Transpose Convolutional layers with calculated padding for "same" effect
        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim,
                                kernel_size=(10 if seq_len > 10 else seq_len), stride=1,
                                padding=(5 if seq_len > 10 else seq_len // 2)),
                nn.Sigmoid()
            )

    def forward(self, z: Tensor) -> Tensor:
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

In [21]:
#| export
def get_conv5_legit_tsgm_vae_components(seq_len, feat_dim, latent_dim, dropout_rate=0.2):
    """
    Returns an instance of Conv5Encoder and Conv5Decoder based on the given parameters.
    
    Args:
        seq_len (int): Length of input sequence.
        feat_dim (int): Dimensionality of input features.
        latent_dim (int): Dimensionality of the latent space.
        dropout_rate (float): Dropout rate to use in the model.
    
    Returns:
        encoder (Conv5Encoder): The encoder part of the VAE.
        decoder (Conv5Decoder): The decoder part of the VAE.
    """
    encoder = Conv5EncoderLegitTsgm(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5DecoderLegitTsgm(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

In [22]:
#| hide
import nbdev; nbdev.nbdev_export()